In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from bingo.optimizer.tf_adam_optimizer import AdamOptimizer
from bingo.gate_sets import ECDGateSet
from bingo.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

2023-02-26 16:58:25.935429: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 16:58:26.011159: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



Need tf version 2.3.0 or later. Using tensorflow version: 2.10.0



ModuleNotFoundError: No module named 'QOGS'

In [9]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'beta_scale' : 1.0, #maximum |beta| for random initialization  
}
ECD_gate_set = ECDGateSet(**gate_set_params)

In [4]:
#The target oscillator state.
fock = 1
psi_t = qt.basis(N,fock) #target state

In [5]:
#the optimization options
synth_params = {
    'gateset' : ECD_gate_set,
    'N_blocks':6,
    'N_multistart' : 100, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [6]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

N_blocks: 6
optimization_type: state transfer
optimization_masks: {'betas_rho': None, 'betas_angle': None, 'phis': None, 'etas': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 100
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: []
do_prints: True
beta_scale: 3.0
filename: Fock 1.h5

Best circuit parameters found:
betas:    tf.Tensor(
[-0.1643 -2.58888j -0.00317-0.01064j  0.61617+1.92017j -1.49207+0.83361j
 -1.00905+2.67741j -0.71373+0.81384j], shape=(6,), dtype=complex64)
phis:    tf.Tensor([ 0.0165   1.05139 -1.51982  1.74225 -3.06771 -1.32724], shape=(6,), dtype=float32)
etas:    tf.Tensor([ 3.08698  2.05205 -0.21682 -2.06456 -3.05512  1.13359], shape=(6,), dtype=float32)
thetas:    tf.Tensor([-2.77055  0.33222  2.23685  0.08906 -1.85906  3.07291], shape=(6,), dtype=float32)

 Best circuit Fidelity: 0.286582




In [7]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-09-09 19:50:36
 Epoch: 9 / 100 Max Fid: 0.997844 Avg Fid: 0.815547 Max dFid: 0.156950 Avg dFid: 0.046593 Elapsed time: 0:00:02.893301 Expected remaining time: 0:00:29.254493

 Optimization stopped. Term fidelity reached.

N_blocks: 6
optimization_type: state transfer
optimization_masks: {'betas_rho': None, 'betas_angle': None, 'phis': None, 'etas': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 100
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: ['2022-09-09 19:50:36']
do_prints: True
beta_scale: 3.0
filename: Fock 1.h5

Best circuit parameters found:
betas:    tf.Tensor(
[-0.58016-1.07921j -0.4053 +0.68378j  0.72741+0.43834j  0.07378+0.38559j
 -0.03577+0.91321j  0.04455+0.8253j ], shape=(6,), dtype=complex64)
phis:    tf.Tensor([-0.99996 -2.62713  1.5283   2.32521  0.96461 -1.78703], shape=(6,), dtype=f

'2022-09-09 19:50:36'

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)